In [ ]:
import pandas as pd
import processor as pr
import loader as load
import predictor as pred

ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)
ge_genus_stage = load.loadGEOverlappingTCMA("genus", includeStage=True)
# aak_ge_stage = load.loadGEWithClinical(includeStage=True)

In [ ]:

print(aak_ge_stage["stage"].value_counts(dropna=False))
d = pred.attachStageStatus(aak_ge_stage)
x, y = pr.splitData(d, "stage", project="STAD")
print(y.value_counts())

print(pred.runCrossValidation(x, y, splits=3, model="MLGR"))

In [ ]:

tcma_genus = load.loadTCMA("genus")
tcma_genus_aak_ge = load.loadGEOverlappingTCMA("genus")
# aak_ge = load.loadGEWithClinical()
aak_ge = None
data = [tcma_genus, aak_ge, tcma_genus_aak_ge]
files = ["tcma_gen", "aak_ge", "tcma_gen_aak_ge"]
# data, files = load.loadAll(with_stage=True)
# print(len(tcma_genus[tcma_genus["project"]=="COAD"]))

In [ ]:
tcma_genus_aak_ge = load.loadGEOverlappingTCMA("genus", includeStage=True)
aak_ge = load.loadGEWithClinical(includeStage=True)
stage_data = [aak_ge, tcma_genus_aak_ge]
pred.runExperiments(stage_data[1:], files[2:], "stage")

In [ ]:

for i, d in enumerate(data[:]):
    d = load.attachTumorStatus(d)
    final_reports = None
    for j, c in enumerate(["COAD", "ESCA", "HNSC", "READ", "STAD"]):   
        for p in [0, 5, 10]:
            if i == 2 and  c == "READ":
                continue
            x, y = pr.splitData(d, target="tumor", project=c)
            
            # print("Before selection", x)
            best_x = pr.selectFeatures(x, y, p)
            x = best_x
            # print("After selection", x)

            cur_report = pred.runCrossValidation(x, y)

            cur_report["cancer"] = c
            cur_report["cv_folds"] = 2
            cur_report["p"] = p

            # Convert elements to array to avoid issues with lack of index when using scaler values from dictionary
            cur_report = {k : [cur_report[k]] for k in cur_report}
            report_d = pd.DataFrame.from_dict(cur_report, orient="columns")

            if final_reports is not None:
                final_reports = pd.concat([final_reports, report_d], ignore_index=True)
            else:
                final_reports = report_d


    output = StringIO()
    final_reports = final_reports.round(3)
    final_reports.to_csv(output, index=False)
    output.seek(0)
    pt = prettytable.from_csv(output)
    print(pt)
    print(final_reports)
    load.saveDescriptor(pt, fr'Data\Descriptor\Prediction_Tables\{files[i]}_tmr_pred.txt')
# Warnings because everything is being predicted as the same
